In [1]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from wordcloud import WordCloud

import re
import contractions

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

# from ydata_profiling import ProfileReport

import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
airbnb_listings = pd.read_csv('airbnb_listing_data_scrape.csv')

In [5]:
airbnb_listings.head()

,Unnamed: 0,web-scraper-start-url,id_x,web_scraper_order,airbnb_listing_url,listing_name,listing_description,listing_hostname,listing_type,listing_room_type,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,listing_id,id_y,date,reviewer_id,reviewer_name,comments
0,0,https://www.airbnb.ca/s/Ontario--Canada/homes?...,1419.0,5.131801e+09,https://www.airbnb.com/rooms/1419,Beautiful home in amazing area!,"This large, family home is located in one of T...",Alexandra,Entire home,Entire home/apt,10.0,3 baths,5.0,NaN,NaN,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1419.0,38924112.0,2015-07-19,11308465.0,Marcela,Having the opportunity of arriving to Alexandr...
1,1,https://www.airbnb.ca/s/Ontario--Canada/homes?...,1419.0,5.131801e+09,https://www.airbnb.com/rooms/1419,Beautiful home in amazing area!,"This large, family home is located in one of T...",Alexandra,Entire home,Entire home/apt,10.0,3 baths,5.0,NaN,NaN,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1419.0,44791978.0,2015-08-29,9580285.0,Marco,We have no enough words to describe how beauty...
2,<br/>We loved this home as it if were our. Th...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,https://www.airbnb.ca/s/Ontario--Canada/homes?...,1419.0,5.131801e+09,https://www.airbnb.com/rooms/1419,Beautiful home in amazing area!,"This large, family home is located in one of T...",Alexandra,Entire home,Entire home/apt,10.0,3 baths,5.0,NaN,NaN,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1419.0,45957133.0,2015-09-07,38394721.0,Andrea,The listing was exceptional and an even better...
4,3,https://www.airbnb.ca/s/Ontario--Canada/homes?...,1419.0,5.131801e+09,https://www.airbnb.com/rooms/1419,Beautiful home in amazing area!,"This large, family home is located in one of T...",Alexandra,Entire home,Entire home/apt,10.0,3 baths,5.0,NaN,NaN,6.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,1419.0,67295154.0,2016-03-28,3515044.0,Shaun,Alexandra's home was amazing and in such a nea...


In [6]:
airbnb_listings.columns

Index(['Unnamed: 0', 'web-scraper-start-url', 'id_x', 'web_scraper_order',
       'airbnb_listing_url', 'listing_name', 'listing_description',
       'listing_hostname', 'listing_type', 'listing_room_type',
       'total_accomodation', 'bathrooms_description', 'total_bedrooms',
       'total_beds', 'listing_price', 'total_reviews', 'overall_rating',
       'accuracy_rating', 'cleanliness_rating', 'checkin_rating',
       'communication_rating', 'location_rating', 'value_rating', 'listing_id',
       'id_y', 'date', 'reviewer_id', 'reviewer_name', 'comments'],
      dtype='object')

In [7]:
airbnb_listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520622 entries, 0 to 520621
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             520622 non-null  object 
 1   web-scraper-start-url  518252 non-null  object 
 2   id_x                   518252 non-null  float64
 3   web_scraper_order      518252 non-null  float64
 4   airbnb_listing_url     518252 non-null  object 
 5   listing_name           518252 non-null  object 
 6   listing_description    512810 non-null  object 
 7   listing_hostname       518250 non-null  object 
 8   listing_type           518252 non-null  object 
 9   listing_room_type      518252 non-null  object 
 10  total_accomodation     518252 non-null  float64
 11  bathrooms_description  517902 non-null  object 
 12  total_bedrooms         492971 non-null  float64
 13  total_beds             432819 non-null  float64
 14  listing_price          431636 non-nu

In [8]:
airbnb_listings = airbnb_listings.sample(n=5000, random_state=42)

In [9]:
airbnb_listings.to_csv('sampled_dataset_airbnb_listings.csv')

In [10]:
columns_to_drop = ['Unnamed: 0', 'web-scraper-start-url', 'web_scraper_order', 'listing_hostname', 'date', 'id_x', 'id_y', 'reviewer_id', 'reviewer_name', 'airbnb_listing_url']
airbnb_listings.drop(columns=columns_to_drop, inplace=True)

In [11]:
airbnb_listings.shape

(5000, 19)

In [12]:
airbnb_listings = airbnb_listings[['listing_id', 'listing_name', 'listing_description',
       'listing_type', 'listing_room_type', 'total_accomodation',
       'bathrooms_description', 'total_bedrooms', 'total_beds',
       'listing_price', 'total_reviews', 'overall_rating', 'accuracy_rating',
       'cleanliness_rating', 'checkin_rating', 'communication_rating',
       'location_rating', 'value_rating',
       'comments']]
airbnb_listings.head()

,listing_id,listing_name,listing_description,listing_type,listing_room_type,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,comments
366230,4.787163e+07,Lakeview & Gorgeous Condo in Heart of Downtown,Our building is located in the heart of the vi...,Private room in condo,Private room,2.0,1 private bath,1.0,1.0,$132.00,190.0,4.95,4.83,4.91,4.91,4.93,4.95,4.74,Mohammadreza was very polite professional a gr...
336984,4.052493e+07,Bright Studio Located Downtown Toronto,"This bright and beautiful studio, located just...",Entire condo,Entire home/apt,2.0,1 bath,1.0,NaN,NaN,45.0,4.78,4.80,4.62,4.89,4.84,4.89,4.78,Great host who welcomed us at the door and sho...
471064,8.121417e+17,Designer Condo 2BR+1BA Sleeps 5 W/ CN Tower Views,Welcome to your home away from home! This mode...,Entire condo,Entire home/apt,5.0,1 bath,2.0,3.0,$79.00,41.0,4.68,4.78,4.76,4.83,4.90,4.93,4.41,"Not a true 2 bedroom, wouldn’t be a problem fo..."
419409,6.255050e+17,🔥Modern High Floor Deluxe King Suite /w City V...,The perfect getaway in the heart of downtown T...,Entire condo,Entire home/apt,3.0,1 bath,1.0,2.0,$288.00,96.0,4.92,4.92,4.90,4.76,4.98,4.90,4.67,Fadi was great and incredibly responsive. The ...
234834,2.516244e+07,"Newly Renovated Unit, in Downtown Toronto!",This perfect newly renovated unit is located i...,Entire home,Entire home/apt,2.0,1 bath,1.0,1.0,$69.00,216.0,4.51,4.58,4.71,4.74,4.81,4.58,4.50,Great location. Great communication from hosts...


In [13]:
airbnb_listings.isnull()

,listing_id,listing_name,listing_description,listing_type,listing_room_type,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,comments
366230,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
336984,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
471064,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
419409,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
234834,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
337792,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False
83979,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
30860,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
140011,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [14]:
airbnb_listings.isnull().sum()

listing_id                83
listing_name              27
listing_description       78
listing_type              27
listing_room_type         27
total_accomodation        27
bathrooms_description     29
total_bedrooms           285
total_beds               854
listing_price            865
total_reviews             27
overall_rating            83
accuracy_rating           83
cleanliness_rating        83
checkin_rating            83
communication_rating      83
location_rating           83
value_rating              83
comments                  83
dtype: int64

In [15]:
# dropping rows with missing values
airbnb_listings.dropna(inplace=True)

In [16]:
airbnb_listings.duplicated().any()

False

In [17]:
airbnb_listings.duplicated().sum()

0

In [18]:
airbnb_listings[airbnb_listings.duplicated()]

,listing_id,listing_name,listing_description,listing_type,listing_room_type,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,comments


In [19]:
# run this to remove duplicates first
airbnb_listings.drop_duplicates(inplace=True)

In [20]:
# final data shape before further pre-processing
airbnb_listings.shape

(4050, 19)

In [21]:
def change_datatype_to_int(column_name):
    try:
        airbnb_listings[column_name] = airbnb_listings[column_name].astype(int)
    except Exception as e:
        print("Something went wrong!", e)

In [22]:
airbnb_listings.columns

Index(['listing_id', 'listing_name', 'listing_description', 'listing_type',
       'listing_room_type', 'total_accomodation', 'bathrooms_description',
       'total_bedrooms', 'total_beds', 'listing_price', 'total_reviews',
       'overall_rating', 'accuracy_rating', 'cleanliness_rating',
       'checkin_rating', 'communication_rating', 'location_rating',
       'value_rating', 'comments'],
      dtype='object')

In [23]:
airbnb_listings['bathrooms_description'].unique()

array(['1 private bath', '1 bath', '4 baths', '2 baths', '1 shared bath',
       '1.5 shared baths', '3 baths', '1.5 baths', '2 shared baths',
       '2.5 shared baths', '5.5 baths', '3.5 baths', '3 shared baths',
       '2.5 baths', '3.5 shared baths', '4.5 shared baths', 'Half-bath',
       '4.5 baths', 'Shared half-bath', '0 shared baths', '6.5 baths',
       '6 baths'], dtype=object)

In [24]:
airbnb_listings['bathrooms_description'].value_counts()

bathrooms_description
1 bath              1965
1 private bath       688
1 shared bath        533
2 baths              427
1.5 baths            147
1.5 shared baths      75
2.5 baths             64
2 shared baths        60
3 baths               26
3.5 baths             21
3 shared baths        12
2.5 shared baths       8
4 baths                4
4.5 baths              4
5.5 baths              3
3.5 shared baths       2
4.5 shared baths       2
Half-bath              2
Shared half-bath       2
0 shared baths         2
6 baths                2
6.5 baths              1
Name: count, dtype: int64

In [25]:
def remove_non_digits_characters(column_name):
    try:
        # Replace non-digit characters with an empty string
        airbnb_listings[column_name] = airbnb_listings[column_name].str.replace('[^\d.]+', '', regex=True)
        if column_name == 'bathrooms_description':
            # Replace empty strings with NaN
            airbnb_listings[column_name].replace('', 0.5, inplace=True)
        # Convert to float
        airbnb_listings[column_name] = airbnb_listings[column_name].astype(float)
    except Exception as e:
        print("Something went wrong!", e)

In [26]:
remove_non_digits_characters('bathrooms_description')

In [27]:
remove_non_digits_characters('listing_price')

In [28]:
change_datatype_to_int('listing_id')
change_datatype_to_int('total_accomodation')
change_datatype_to_int('total_bedrooms')
change_datatype_to_int('total_beds')
change_datatype_to_int('total_reviews')

In [29]:
airbnb_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4050 entries, 366230 to 357812
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   listing_id             4050 non-null   int64  
 1   listing_name           4050 non-null   object 
 2   listing_description    4050 non-null   object 
 3   listing_type           4050 non-null   object 
 4   listing_room_type      4050 non-null   object 
 5   total_accomodation     4050 non-null   int64  
 6   bathrooms_description  4050 non-null   float64
 7   total_bedrooms         4050 non-null   int64  
 8   total_beds             4050 non-null   int64  
 9   listing_price          4050 non-null   float64
 10  total_reviews          4050 non-null   int64  
 11  overall_rating         4050 non-null   float64
 12  accuracy_rating        4050 non-null   float64
 13  cleanliness_rating     4050 non-null   float64
 14  checkin_rating         4050 non-null   float64
 15  co

In [30]:
airbnb_listings.head()

,listing_id,listing_name,listing_description,listing_type,listing_room_type,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,comments
366230,47871632,Lakeview & Gorgeous Condo in Heart of Downtown,Our building is located in the heart of the vi...,Private room in condo,Private room,2,1.0,1,1,132.0,190,4.95,4.83,4.91,4.91,4.93,4.95,4.74,Mohammadreza was very polite professional a gr...
471064,812141696294837504,Designer Condo 2BR+1BA Sleeps 5 W/ CN Tower Views,Welcome to your home away from home! This mode...,Entire condo,Entire home/apt,5,1.0,2,3,79.0,41,4.68,4.78,4.76,4.83,4.90,4.93,4.41,"Not a true 2 bedroom, wouldn’t be a problem fo..."
419409,625504954174240640,🔥Modern High Floor Deluxe King Suite /w City V...,The perfect getaway in the heart of downtown T...,Entire condo,Entire home/apt,3,1.0,1,2,288.0,96,4.92,4.92,4.90,4.76,4.98,4.90,4.67,Fadi was great and incredibly responsive. The ...
234834,25162440,"Newly Renovated Unit, in Downtown Toronto!",This perfect newly renovated unit is located i...,Entire home,Entire home/apt,2,1.0,1,1,69.0,216,4.51,4.58,4.71,4.74,4.81,4.58,4.50,Great location. Great communication from hosts...
367474,48752454,Luxury 3 BR semi with free parking (Entire House),This amazing newly built custom semi-detached ...,Entire home,Entire home/apt,7,4.0,3,3,409.0,34,4.79,4.88,4.56,5.00,5.00,4.88,4.76,Fantastic place in a relaxing and quant neighb...


In [31]:
class TextPreprocessor:
    def __init__(self):
        """
        Initialize the TextPreprocessor class.
        Set up stop words and lemmatizer.
        """
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

    def lowercase(self, text):
        """
        Convert text to lowercase.

        Parameters:
        - text (str): Input text.

        Returns:
        - str: Lowercased text.
        """
        return str(text).lower()

    def remove_urls(self, text):
        """
        Remove URLs from text.

        Parameters:
        - text (str): Input text.

        Returns:
        - str: Text with URLs removed.
        """
        # return re.sub(r'http\S+', '', text)
        return re.sub(r'https?://\S+', '', text)

    def remove_non_word_non_whitespace(self, text):
        """
        Remove non-word and non-whitespace characters from text.

        Parameters:
        - text (str): Input text.

        Returns:
        - str: Text with non-word and non-whitespace characters removed.
        """
        return re.sub(r'[^\w\s]', '', text)

    def remove_digits(self, text):
        """
        Remove digits from text.

        Parameters:
        - text (str): Input text.

        Returns:
        - str: Text with digits removed.
        """
        return re.sub(r'\d+', '', text)

    def tokenize(self, text):
        """
        Tokenize text into words.

        Parameters:
        - text (str): Input text.

        Returns:
        - list: List of tokens (words).
        """
        return word_tokenize(text)

    def remove_stopwords(self, tokens):
        """
        Remove stopwords from tokens.

        Parameters:
        - tokens (list): List of tokens.

        Returns:
        - list: List of tokens with stopwords removed.
        """
        return [word for word in tokens if word not in self.stop_words]

    def lemmatize(self, tokens):
        """
        Lemmatize tokens.

        Parameters:
        - tokens (list): List of tokens.

        Returns:
        - list: List of lemmatized tokens.
        """
        return [self.lemmatizer.lemmatize(word) for word in tokens]

    def expand_contractions(self, text):
        """
        Expand contractions in text.

        Parameters:
        - text (str): Input text.

        Returns:
        - str: Text with contractions expanded.
        """
        return contractions.fix(text)

    def preprocess_text(self, text):
        """
        Preprocess text using various techniques.

        Parameters:
        - text (str): Input text.

        Returns:
        - str: Preprocessed text.
        """

        # Applying preprocessing steps
        text = self.lowercase(text)
        text = self.remove_urls(text)
        text = self.remove_non_word_non_whitespace(text)
        text = self.remove_digits(text)
        text = self.expand_contractions(text)
        tokens = self.tokenize(text)
        tokens = self.remove_stopwords(tokens)
        tokens = self.lemmatize(tokens)
        return ' '.join(tokens)

In [32]:
preprocessor = TextPreprocessor()

In [33]:
def preprocess_text(text_column):
    """
    Preprocess text data in the DataFrame.

    This function applies text preprocessing using the TextPreprocessor object
    to the specified text column in the DataFrame.

    """
    try:
        # applying text preprocessing to the specified text column
        airbnb_listings[f'processed_text_{text_column}'] = airbnb_listings[text_column].apply(preprocessor.preprocess_text)
    except Exception as e:
        # handleing exceptions during preprocessing
        print("Something went wrong with the text data: ", e)

In [34]:
text_columns = ['listing_name', 'listing_description', 'listing_type', 'listing_room_type', 'comments']
for column in text_columns:
    preprocess_text(column)

In [35]:
airbnb_listings.drop(columns=text_columns, inplace=True)

In [36]:
airbnb_listings.head()

,listing_id,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,processed_text_listing_name,processed_text_listing_description,processed_text_listing_type,processed_text_listing_room_type,processed_text_comments
366230,47871632,2,1.0,1,1,132.0,190,4.95,4.83,4.91,4.91,4.93,4.95,4.74,lakeview gorgeous condo heart downtown,building located heart vibrant cityplace neigh...,private room condo,private room,mohammadreza polite professional great host lo...
471064,812141696294837504,5,1.0,2,3,79.0,41,4.68,4.78,4.76,4.83,4.90,4.93,4.41,designer condo brba sleep w cn tower view,welcome home away home modern designed condo s...,entire condo,entire homeapt,true bedroom would problem kid ended adjust lo...
419409,625504954174240640,3,1.0,1,2,288.0,96,4.92,4.92,4.90,4.76,4.98,4.90,4.67,modern high floor deluxe king suite w city view,perfect getaway heart downtown torontobr br ra...,entire condo,entire homeapt,fadi great incredibly responsive place great w...
234834,25162440,2,1.0,1,1,69.0,216,4.51,4.58,4.71,4.74,4.81,4.58,4.50,newly renovated unit downtown toronto,perfect newly renovated unit located downtown ...,entire home,entire homeapt,great location great communication host troubl...
367474,48752454,7,4.0,3,3,409.0,34,4.79,4.88,4.56,5.00,5.00,4.88,4.76,luxury br semi free parking entire house,amazing newly built custom semidetached storey...,entire home,entire homeapt,fantastic place relaxing quant neighborhood co...


In [37]:
airbnb_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4050 entries, 366230 to 357812
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   listing_id                          4050 non-null   int64  
 1   total_accomodation                  4050 non-null   int64  
 2   bathrooms_description               4050 non-null   float64
 3   total_bedrooms                      4050 non-null   int64  
 4   total_beds                          4050 non-null   int64  
 5   listing_price                       4050 non-null   float64
 6   total_reviews                       4050 non-null   int64  
 7   overall_rating                      4050 non-null   float64
 8   accuracy_rating                     4050 non-null   float64
 9   cleanliness_rating                  4050 non-null   float64
 10  checkin_rating                      4050 non-null   float64
 11  communication_rating                4050 

In [38]:
airbnb_listings.describe()

,listing_id,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating
count,4.050000e+03,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000,4050.000000
mean,1.884584e+17,3.183457,1.221605,1.331111,1.698272,159.217531,142.605432,4.824449,4.861733,4.805432,4.890111,4.909751,4.853326,4.762951
std,3.386290e+17,1.935404,0.510415,0.827261,1.081432,263.750650,136.310137,0.150589,0.129905,0.187796,0.124881,0.114825,0.138963,0.163858
min,2.665400e+04,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.000000,3.000000,3.000000,3.560000,3.000000,3.000000,2.000000
25%,1.630031e+07,2.000000,1.000000,1.000000,1.000000,78.000000,44.000000,4.760000,4.820000,4.730000,4.860000,4.880000,4.800000,4.690000
50%,3.015635e+07,2.000000,1.000000,1.000000,1.000000,116.000000,98.000000,4.860000,4.890000,4.850000,4.930000,4.940000,4.890000,4.790000
75%,5.398345e+07,4.000000,1.000000,2.000000,2.000000,185.000000,201.000000,4.930000,4.950000,4.940000,4.970000,4.980000,4.950000,4.860000
max,1.065418e+18,16.000000,6.500000,7.000000,11.000000,10000.000000,1035.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [39]:
# word cloud of the combined text columns
# distribution plot for all the numerical columns
# box plot for all the numerical columns

In [40]:
# text_columns = airbnb_listings.select_dtypes(include=['object']).columns
# numeric_columns = airbnb_listings.select_dtypes(include=['number']).columns

In [41]:
text_columns = ['processed_text_listing_name', 'processed_text_listing_description', 'processed_text_listing_type', 'processed_text_listing_room_type', 'processed_text_comments']
def combine_text_columns(text_columns):
    airbnb_listings['combined_text'] = airbnb_listings[text_columns].apply(lambda x: ' '.join(x), axis=1) # rename to reviews

combine_text_columns(text_columns)

In [42]:
airbnb_listings.drop(columns=text_columns, inplace=True)

In [43]:
airbnb_listings.head()

,listing_id,total_accomodation,bathrooms_description,total_bedrooms,total_beds,listing_price,total_reviews,overall_rating,accuracy_rating,cleanliness_rating,checkin_rating,communication_rating,location_rating,value_rating,combined_text
366230,47871632,2,1.0,1,1,132.0,190,4.95,4.83,4.91,4.91,4.93,4.95,4.74,lakeview gorgeous condo heart downtown buildin...
471064,812141696294837504,5,1.0,2,3,79.0,41,4.68,4.78,4.76,4.83,4.90,4.93,4.41,designer condo brba sleep w cn tower view welc...
419409,625504954174240640,3,1.0,1,2,288.0,96,4.92,4.92,4.90,4.76,4.98,4.90,4.67,modern high floor deluxe king suite w city vie...
234834,25162440,2,1.0,1,1,69.0,216,4.51,4.58,4.71,4.74,4.81,4.58,4.50,newly renovated unit downtown toronto perfect ...
367474,48752454,7,4.0,3,3,409.0,34,4.79,4.88,4.56,5.00,5.00,4.88,4.76,luxury br semi free parking entire house amazi...


In [44]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(airbnb_listings['combined_text'])

In [45]:
# def vectorize_text(text_column):
#     """
#     Vectorize text data using different methods.

#     This function performs text vectorization using either TF-IDF Vectorization
#     or Bag of Words (BoW) Vectorization, based on the value of the TF_VECTOR parameter.

#     Returns:
#     - tuple: Tuple containing the vectorized matrix and the corresponding vectorizer object.

#     """
#     # initializing TF-IDF Vectorizer with max_features=5000
#     # vectorizing text data using TF-IDF Vectorizer
#     tfidf_matrix = tfidf_vectorizer.fit_transform(airbnb_listings[text_column])
#     return tfidf_matrix

In [46]:
# tfidf_matrix = vectorize_text('combined_text')

In [47]:
# numerical_features = ['total_accomodation', 'bathrooms_description', 'total_bedrooms', 'total_beds', 'listing_price', 'total_reviews', 'overall_rating', 'accuracy_rating', 'cleanliness_rating', 'checkin_rating', 'communication_rating', 'location_rating', 'value_rating']
# X_numerical = airbnb_listings[numerical_features]

In [48]:
tfidf_matrix.shape

(4050, 14378)

In [ ]:
numerical_features_matrix = airbnb_listings[numerical_features].values[:tfidf_matrix.shape[0], :]

In [52]:
from scipy.sparse import hstack
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Normalize numeric columns
scaler = MinMaxScaler()
# airbnb_listings[numerical_features] = scaler.fit_transform(airbnb_listings[numerical_features])

# Combine TF-IDF matrix with normalized numeric columns
tfidf_matrix_combined = hstack((tfidf_matrix, numerical_features_matrix), format='csr')

In [ ]:
# Combine numerical and textual features
# X_combined = hstack((X_numerical, tfidf_matrix))

In [ ]:
tfidf_matrix_combined.shape

In [50]:
user_query_vector.shape

(1, 14378)

In [49]:
# User search query
user_query = 'beautiful apartment near beach'
# pre-process it
# tfidf_vectorizer = TfidfVectorizer()
user_query_vector = tfidf_vectorizer.transform([user_query])

In [58]:
# Content-Based Filtering: Cosine Similarity [working but I wanna try next one]
content_similarity = cosine_similarity(user_query_vector, tfidf_matrix)

In [59]:
print(content_similarity)

[[0.03285253 0.         0.         ... 0.02845067 0.03291032 0.        ]]


In [60]:
# Content-Based Recommendation
# Get indices of most similar listings
content_indices = content_similarity.argsort()[0][-10:][::-1]

# Get top recommendations
content_recommendations = airbnb_listings.iloc[content_indices]

In [61]:
content_recommendations[['combined_text']]

,combined_text
424583,alessias beach house bedroom basement suite un...
28765,beach suite queen st boardwalk beach boardwalk...
424577,alessias beach house bedroom basement suite un...
384105,bdrm boho chic home beach delightful immaculat...
427597,beautiful single bedroom upper beach comfortab...
32640,studio near beach toronto free parking better ...
270213,newly renovated cozy apartment beach enjoy sta...
384139,bdrm boho chic home beach delightful immaculat...
270224,newly renovated cozy apartment beach enjoy sta...
287639,comfy apartment step beach welcome beach enjoy...


In [62]:
from sklearn.neighbors import NearestNeighbors

In [63]:
# Collaborative Filtering: K-nearest neighbors
k = 10  # Number of neighbors to consider
knn_model = NearestNeighbors(metric='cosine', n_neighbors=k, algorithm='brute')
knn_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [64]:
# Collaborative Filtering Recommendation
_, knn_indices = knn_model.kneighbors(user_query_vector, n_neighbors=10)
knn_recommendations = airbnb_listings.iloc[knn_indices[0]]

In [65]:
knn_recommendations[['combined_text']]

,combined_text
424583,alessias beach house bedroom basement suite un...
28765,beach suite queen st boardwalk beach boardwalk...
424577,alessias beach house bedroom basement suite un...
384105,bdrm boho chic home beach delightful immaculat...
427597,beautiful single bedroom upper beach comfortab...
32640,studio near beach toronto free parking better ...
270213,newly renovated cozy apartment beach enjoy sta...
384139,bdrm boho chic home beach delightful immaculat...
270224,newly renovated cozy apartment beach enjoy sta...
287639,comfy apartment step beach welcome beach enjoy...


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

# Pivot the data into a user-item matrix
user_item_matrix = airbnb_listings.pivot(index='listing_id', columns='user_id', values='rating').fillna(0)

# Apply Singular Value Decomposition (SVD)
svd = TruncatedSVD(n_components=50, random_state=42)
svd.fit(user_item_matrix)

# User search query
user_query = "cozy apartment near downtown"

# Preprocess user input
user_query = preprocess_user_query(user_query)

# Transform user query to the latent space
user_query_vector = svd.transform(user_item_matrix.loc[:, user_query])

# Compute cosine similarity between user query vector and item vectors
similarities = cosine_similarity(user_query_vector, svd.components_)

# Get indices of most similar items
indices = similarities.argsort()[0][-10:][::-1]

# Get recommended listings
recommended_listings = data[data['listing_id'].isin(user_item_matrix.iloc[:, indices].index)]
print("Collaborative Filtering Recommendations:")
print(recommended_listings)
